![](Images\2022-11-26-09-08-29.png)

### Define using Abstract Model
coef.dat
![](Images\2022-11-26-09-09-58.png)


In [3]:
# create params.dat file    # we define the sets with capital letters
params_file = """

param : I : c :=
1 7
2 12;

param : M : b :=
1 4
2 2;

param a :=
1 1 3
1 2 4
2 1 2
2 2 5;

"""
with open('07_params.dat', 'w') as file:
    file.write(params_file)

In [27]:
# Another way to define a_index

# create params.dat file    # we define the sets with capital letters

params_file = """

param : I : c :=
        1   7
        2   12;

param : M : b :=
        1   4
        2   2;

param a:  1 2 :=
        1 3 4
        2 2 5;

"""
with open('07_params.dat', 'w') as file:
        file.write(params_file)

In [28]:
import pyomo.environ as pyo
import numpy as np

In [29]:
# Initiate the model (Abstract Model)
model = pyo.AbstractModel()

# Declare the parameters and variables
model.I = pyo.Set()
model.M = pyo.Set()

model.c = pyo.Param(model.I)
model.b = pyo.Param(model.M)
model.a = pyo.Param(model.M, model.I)

model.x = pyo.Var(model.I, within= pyo.NonNegativeReals )

# Declare the constraints
def constraint_1(model, m):
    return np.sum(model.a[m, i] * model.x[i] for i in model.I) >= model.b[m]
model.con1 = pyo.Constraint(model.M, rule=constraint_1)

# Declare the objective function
def objective_1(model):
    return np.sum(model.c[i] * model.x[i] for i in model.I)
model.OF = pyo.Objective(rule=objective_1, sense=pyo.minimize)

# initiate the solver
solver = pyo.SolverFactory('glpk')
# create instance from abstract model
instance = model.create_instance('07_params.dat')
# solve the problem
results = solver.solve(instance)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 9.33333333333333
  Upper bound: 9.33333333333333
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 3
  Number of nonzeros: 5
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.055290937423706055
# ----------------------------------------------------------
#   Solution Information
# --------------

C:\Users\20100\AppData\Local\Temp\ipykernel_11620\3989363460.py:16: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(model.a[m, i] * model.x[i] for i in model.I) >= model.b[m]
C:\Users\20100\AppData\Local\Temp\ipykernel_11620\3989363460.py:21: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(model.c[i] * model.x[i] for i in model.I)


In [30]:
instance.pprint()

3 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    M : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    a_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    M*I :    4 : {(1, 1), (1, 2), (2, 1), (2, 2)}

3 Param Declarations
    a : Size=4, Index=a_index, Domain=Any, Default=None, Mutable=False
        Key    : Value
        (1, 1) :     3
        (1, 2) :     4
        (2, 1) :     2
        (2, 2) :     5
    b : Size=2, Index=M, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :     4
          2 :     2
    c : Size=2, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :     7
          2 :    12

1 Var Declarations
    x : Size=2, Index=I
        Key : Lower : Value        

In [31]:
# print the results
for i in instance.I:
    print(f"X_{i} : ", pyo.value(instance.x[i]))

print('OF: ', pyo.value(instance.OF))

X_1 :  1.33333333333333
X_2 :  0.0
OF:  9.333333333333309
